In [4]:
import random
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
import vectorbtpro as vbt
from vctr.models.lstm_keras import LSTM_Model

coins = [
    'AVT',
    'AXS',
    'BADGER',
    'BAL',
    'BAND',
    'BAT',
    'BOSON',
    'BSV',
    'BTG',
    'BTRST',
    'BTSE',
    'BTT',
    'CLV',
    'COMP',
    'COTI',
    'COVAL',
    'CRO',
    'CRPT',
    'CRV',
    'CTSI',
    'CVC',
    'DAI',
    'DASH',
    'DDX',
    'DGB',
    'DNT',
    'DOGE',
    'DOT',
    'DSH',
    'DUSK',
    'DVF',
    'EDO',
    'EGLD',
    'ELA',
    'ENJ',
    'EOS',
    'ETC',
    'ETH',
    'ETP',
    'FARM',
    'FCL',
    'FET',
    'FIL',
    'FLOW',
    'FORTH',
    'FOX',
    'FTM',
    'FTT',
    'FUN',
    'FX',
    'GALA',
    'GLM',
    'GNO',
    'GNT',
    'GRT',
    'GTC',
    'GYEN',
    'HBAR',
    'HMT',
    'ICE',
    'ICP',
    'ICX',
    'IMX',
    'INJ',
    'IOT',
    'IOTX',
    'IQX',
    'JASMY',
    'JST',
    'KAVA',
    'KEEP',
    'KNC',
    'KRL',
    'KSM',
    'LCX',
    'LINK',
    'LOOM',
    'LPT',
    'LRC',
    'LSK',
    'LTC',
    'LUNA',
    'LYM',
    'MANA',
    'MASK',
    'MATIC',
    'MIR',
    'MKR',
    'MLN',
    'MNA',
    'MOB',
    'NEAR',
    'BTC',
    'NANO',
    'NCT',
    'NEAR',
    'NEO',
    'NEXO',
    'NKN',
    'NMR',
    'NU',
    'OCEAN',
    'OGN',
    'OMG',
    'ORN',
    'OXT',
    'OXY',
    'PAS',
    'PAX',
    'PAXG',
    'PERP',
    'PLA',
    'PLANETS',
    'PNK',
    'POLY',
    'QNT',
    'QRDO',
    'QTF',
    'QTM',
    'QTUM',
    'QUICK',
    'RAI',
    'RARI',
    'REEF',
    'REN',
    'REP',
    'REQ',
    'RLC',
    'RLY',
    'RNDR',
    'ROSE',
    'SAND',
    'SC',
    'SGB',
    'SHIB',
    'SKL',
    'SNT',
    'SNX',
    'SOL',
    'SPELL',
    'SRM',
    'STORJ',
    'STX',
    'SUKU',
    'SUSHI',
    'SWFTC',
    'TLOS',
    'TRAC',
    'TRB',
    'TRIBE',
    'TRU',
    'TRX',
    'TSD',
    'UDC',
    'UMA',
    'UNI',
    'UOS',
    'UTK',
    'VEE',
    'VELO',
    'VET',
    'VGX',
    'VSY',
    'WAVES',
    'WAX',
    'WBTC',
    'XAUT',
    'XDC',
    'XLM',
    'XMR',
    'XRA',
    'XRD',
    'XRP',
    'XTZ',
    'XVG',
    'XYO',
    'YFI',
    'YGG',
    'ZCN',
    'ZEC',
    'ZEN',
    'ZIL',
    'ZMT',
    'ZRX',
]

# sample = random.sample(coins, 5)
sample = ['BTC']

print(sample)


def plot_price_preds(data):
    # data = data[-2000:]

    # Get the max price for the y-axis
    max_price = data['close'].max()

    green_bars = data[data['pred'] == 1]['close']
    red_bars = data[data['pred'] == 2]['close']

    # Create line plot for close prices
    price_trace = go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close Price', line=dict(width=2))

    # Create green and red vertical bar shapes
    green_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=price,
                y1=0,
                opacity=0.3,
                line=dict(width=5, color='#30c77b'),
            )
            for timestamp, price in green_bars.items()
        ),
    ]

    red_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=price,
                y1=max_price * 2,
                opacity=0.3,
                line=dict(width=5, color='#c43338'),
            )
            for timestamp, price in red_bars.items()
        ),
    ]

    # Combine all traces and create a plot
    layout = go.Layout(
        shapes=green_shapes + red_shapes,
        title=dict(text='Price with Buy/Sell Predictions', font=dict(color='white')),
        xaxis=dict(
            title='Timestamp',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        yaxis=dict(
            title='Price',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        plot_bgcolor='rgba(32, 32, 32, 1)',
        paper_bgcolor='rgba(32, 32, 32, 1)',
        legend=dict(font=dict(color='white')),
        width=1200,
        height=700,
    )

    fig = go.Figure(data=[price_trace], layout=layout)
    fig.show()


def plot_price_preds_actual(data):
    # data = data[-2000:]

    # Get the max price for the y-axis
    max_price = data['close'].max()

    # Extract data for green and red bars
    green_bars_actual = data[data['label'] == 1]['close']
    red_bars_actual = data[data['label'] == 2]['close']

    green_bars = data[data['pred'] == 1]['close']
    red_bars = data[data['pred'] == 2]['close']

    # Create line plot for close prices
    price_trace = go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close Price', line=dict(width=1))

    # Create green and red vertical bar shapes
    green_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=0,
                y1=price,
                opacity=0.2,
                line=dict(width=3, color='#30c77b'),
            )
            for timestamp, price in green_bars.items()
        ),
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=price,
                y1=max_price * 2,
                opacity=0.2,
                line=dict(width=3, color='#30c77b'),
            )
            for timestamp, price in green_bars_actual.items()
        ),
    ]

    red_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=0,
                y1=price,
                opacity=0.2,
                line=dict(width=3, color='#c43338'),
            )
            for timestamp, price in red_bars.items()
        ),
        *(
            dict(
                type='rect',
                xref='x',
                yref='y',
                x0=timestamp,
                x1=timestamp,
                y0=price,
                y1=max_price * 2,
                opacity=0.2,
                line=dict(width=3, color='#c43338'),
            )
            for timestamp, price in red_bars_actual.items()
        ),
    ]

    # Combine all traces and create a plot
    layout = go.Layout(
        shapes=green_shapes + red_shapes,
        title=dict(text='Price with Buy/Sell Labels', font=dict(color='white')),
        xaxis=dict(
            title='Timestamp',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        yaxis=dict(
            title='Price',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        plot_bgcolor='rgba(32, 32, 32, 1)',
        paper_bgcolor='rgba(32, 32, 32, 1)',
        legend=dict(font=dict(color='white')),
        width=1200,
        height=700,
    )

    fig = go.Figure(data=[price_trace], layout=layout)
    fig.show()


def get_return(df):
    pf = vbt.Portfolio.from_signals(
        df['close'],
        df['pred'] == 1,
        df['pred'] == 2,
        freq='1H',
        init_cash=10000,
        fees=0.001,
        fixed_fees=0.001,
        slippage=0.001,
        log=True,
    )
    return f'ROI: {pf.stats("total_return").values[0]:.0f}%'


def threshold_predictions(probs, thresholds):
    """
    Given a 2D array of prediction probabilities and an array of thresholds,
    returns an array of class labels.

    The class label for each row defaults to the majority class (0) unless one
    of the minority classes (1 & 2) exceed their threshold. If both exceed their
    threshold, then the class label is whichever one exceeded their threshold by more.
    """
    # Find the index of the majority class (0)
    majority_class_idx = 0

    # Find the indices of the minority classes (1 & 2)
    minority_class_idxs = [1, 2]

    # Initialize the predicted class labels to the majority class (0)
    pred_labels = np.zeros(probs.shape[0])

    # Loop over each row of the probability matrix
    for i in range(probs.shape[0]):
        # Check if either of the minority classes exceed their threshold
        if (
            probs[i, minority_class_idxs[0]] > thresholds[minority_class_idxs[0]]
            or probs[i, minority_class_idxs[1]] > thresholds[minority_class_idxs[1]]
        ):
            # Determine which minority class exceeded its threshold by more
            diff_1 = probs[i, minority_class_idxs[0]] - thresholds[minority_class_idxs[0]]
            diff_2 = probs[i, minority_class_idxs[1]] - thresholds[minority_class_idxs[1]]
            if diff_1 > diff_2:
                pred_labels[i] = 1
            else:
                pred_labels[i] = 2

    return pred_labels


def optimal_thresholds(y_true, y_proba):
    """
    Calculates the optimal classification threshold for each class based
    on what would yield the greatest overlap with y_true.
    :param y_true: actual labels of the data
    :param y_proba: predicted probabilities of each label being true
    :return: list of optimal thresholds for each class
    """

    num_classes = y_true.shape[1]  # number of classes
    thresholds = np.zeros(num_classes)  # initialize empty array

    for i in range(num_classes):  # iterate over all classes
        precision, recall, threshold = metrics.precision_recall_curve(y_true[:, i], y_proba[:, i])
        f1_scores = (2 * precision * recall) / (precision + recall)  # calculate F1 scores

        # get the index of the threshold that maximizes F1 score
        index = np.argmax(f1_scores)

        # set the optimal threshold for this class
        thresholds[i] = threshold[index]

    return thresholds


model_args = dict(name='lstm_871', n_features=240, batch_size=512, n_timesteps=24, label_args=(0.03, 0.005))


def model():
    return LSTM_Model(**model_args)


lstm = model()

lstm.train(sample, '1h')
# df = lstm.predict('NEAR', '30m')
# plot_price_preds(df[-4000:])
# print(get_return(df))
# print(df['label'].value_counts(normalize=True))
# print(df['pred'].value_counts(normalize=True))


['BTC']
Training BTC (1h)...


loss [0.560], rec [0.678], prec [0.705], acc [0.691]: 100%|██████████| 263/263 [00:05<00:00, 47.59batch/s]

ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/keras/engine/training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/keras/engine/training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/keras/engine/training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/keras/engine/training.py", line 1758, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/opt/homebrew/C

loss [0.560], rec [0.678], prec [0.705], acc [0.691]: 100%|██████████| 263/263 [00:20<00:00, 47.59batch/s]

In [ ]:
df = lstm.predict('APE', '1h')
# df = get_data_preds_from_thresh('NEAR', '1h', thresh=[0.33, 0.68, 0.69])

plot_price_preds(df[-4000:])
print(get_return(df[-4000:]))
print(df['label'].value_counts(normalize=True))
print(df['pred'].value_counts(normalize=True))

In [ ]:
def get_data_preds_from_thresh(symbol, timeframe, thresh=[0.33, 0.72, 0.66]):
    X, y_true, y_proba, data = lstm.predict_custom(symbol, timeframe)
    pred = threshold_predictions(y_proba, thresh)
    data = data[len(data) - len(y_true) :]
    data['pred'] = pd.Series(pred, index=data.index)
    # data['label'] = pd.Series(y_true, index=data.index)
    return data

In [ ]:
lstm.save_model()